# 本地环境加载

In [ ]:
import os
from dotenv import load_dotenv
from langchain_qwq import ChatQwen
from langchain.agents import create_agent

load_dotenv()
qwen_api_key = os.getenv("DASHSCOPE_API_KEY")

# 静态模型
静态模型在创建代理时进行一次配置，并在整个执行过程中保持不变。这是最常见且最直接的方法。

In [ ]:
model = ChatQwen(
    model="qwen-flash",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
)
agent = create_agent(model)

ai_msg = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "你好，你是什么模型"
        }
    ]
})

print(ai_msg["messages"][-1].content)
print(ai_msg["messages"][-1].response_metadata["model_name"])
# print(ai_msg["messages"][-1])

# 动态模型
动态模型的选择是在运行时环境基于当前状态以及上下文信息。这使得复杂的路由逻辑和成本优化成为可能。

需要使用@wrap_model_call装饰器创建中间件，该装饰器会在请求中修改模型

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

basic_model = ChatQwen(model="qwen-flash", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", )
advanced_model = ChatQwen(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", )


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """根据对话的复杂程度选择合适的模型。"""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # 对于较长的对话，请使用高级模型。
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))


agent = create_agent(
    model=basic_model,  # Default model
    middleware=[dynamic_model_selection]
)


ai_msg = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "今有雉（鸡）、兔同笼，上有三十五头（总头数35），下有九十四足（总脚数94）。问雉、兔各几何？让我们一步一步思考"
        }
    ]
})

print(ai_msg["messages"][-1].content)
print(ai_msg["messages"][-1].response_metadata["model_name"])